# **Test and Use the model**
I have trained the model in Google Colab, and now we can test whether it can detect TBC or not.

In [1]:
import numpy as np
import tensorflow as tf
import os
from PIL import Image, ImageTk

2025-01-12 18:08:17.441456: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-12 18:08:17.445549: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-12 18:08:17.458059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-12 18:08:17.478264: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-12 18:08:17.483886: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 18:08:17.499367: I tensorflow/core/platform/cpu_feature_gu

# 1. Load and Allocate the Model
Load model to the environment and allocate memory for it.

In [2]:
interpreter = tf.lite.Interpreter(model_path='/home/umar/myenv/SelfProject/TBCMLModels/NEW2_3.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# 2. Get Input & Output from the Model
We should get the architecture and information of input & output within the model.

In [3]:
input_details= interpreter.get_input_details()
output_details= interpreter.get_output_details()

print(input_details)
print(output_details)

[{'name': 'serving_default_keras_tensor_70:0', 'index': 0, 'shape': array([  1, 224, 224,   1], dtype=int32), 'shape_signature': array([ -1, 224, 224,   1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall_1:0', 'index': 49, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


# 3. Create a Function for Predicting

In [4]:
def output_predict(X):
  input_data = X.astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()
  predictions= interpreter.get_tensor(output_details[0]['index']).ravel()

  if predictions[0] < 0.5:
    print('The lung looks good')
    print(f'The percentages of Tuberculosis exist is {predictions[0] * 100:.2f} %')
  else:
    print('There is high chances that the lung have Tuberculosis in it')
    print(f'The percentages of Tuberculosis exist is {predictions[0] * 100:.2f} %')

# 4. Load the picture

In [5]:
img_path = '/home/umar/Documents/Datasets/TB_Chest_Radiography_Database/Test/Tuberculosis/Tuberculosis-2935.png'
img = Image.open(img_path).convert('L')
img= img.resize((224, 224))
img_array= np.array(img)
img_array = img_array.reshape(1, 224, 224, 1)
img_array = img_array/255.0
img_array.shape

(1, 224, 224, 1)

# 5. Test

In [6]:
output_predict(img_array)

There is high chances that the lung have Tuberculosis in it
The percentages of Tuberculosis exist is 98.98 %


# **Create GUI Functionality**

In [13]:
import tkinter as tk
from tkinterdnd2 import DND_FILES, TkinterDnD
from tkinter import filedialog
from tkinter import ttk
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf


def output_predict(X):
    input_data = X.astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_details[0]['index']).ravel()

    if predictions[0] < 0.5:
        return f'The lung looks good\nThe percentage of Tuberculosis present is {predictions[0] * 100:.2f}%'
    else:
        return f'There is a high chance that the lung has Tuberculosis\nThe percentage of Tuberculosis present is {predictions[0] * 100:.2f}%'


def load_image(img_path):
    img = Image.open(img_path).convert('L')
    img = img.resize((224, 224))
    img_array = np.array(img).reshape(1, 224, 224, 1) / 255.0
    return img_array


def process_image(file_path):
    img_array = load_image(file_path)
    result_text = output_predict(img_array)

    # Display the image in grayscale
    img = Image.open(file_path)
    img = img.resize((224, 224))
    img = ImageTk.PhotoImage(img.convert('L'))

    image_label.config(image=img)
    image_label.image = img

    image_frame.pack(pady=10)
    result_label.config(text=result_text)
    result_frame.pack(pady=10)
    back_button.pack(pady=20)  # Show the back button
    drag_drop_frame.pack_forget()  # Hide drag-and-drop interface
    button.pack_forget()  # Hide the file chooser button


def on_drop(event):
    file_path = event.data.strip('{}')  # Get the dragged file path
    process_image(file_path)


def choose_image():
    file_path = filedialog.askopenfilename(filetypes=[("PNG Images", "*.png")])
    if file_path:
        process_image(file_path)


def go_back():
    # Reset the interface
    result_label.config(text="")
    image_label.config(image="")
    image_frame.pack_forget()
    result_frame.pack_forget()
    back_button.pack_forget()  # Hide the back button
    button.pack(pady=20)  # Show the file chooser button
    drag_drop_frame.pack(pady=10)  # Show drag-and-drop interface


# Create the main window
root = TkinterDnD.Tk()  # Use TkinterDnD.Tk instead of tk.Tk
root.title("TB Lung Image Predictor")
root.geometry("500x700")
root.configure(bg="#F8F8F8")  # Light background for elegance

# Apply Style Theme
style = ttk.Style()
style.theme_use('clam')

# Define custom style for the button
style.configure('Custom.TButton',
                font=('Helvetica', 14),
                padding=10,
                borderwidth=0,
                relief="solid",
                foreground="white",
                background="#5A9",
                focuscolor="none")

# Create a title label
title_label = tk.Label(root, text="TB Lung Image Predictor", font=("Helvetica", 18, "bold"), bg="#F8F8F8", fg="#333")
title_label.pack(pady=20)

# Create a frame for the image display
image_frame = tk.Frame(root, bg="#FFFFFF", relief="groove", bd=2)
image_label = ttk.Label(image_frame, background="#FFFFFF")
image_label.pack(padx=10, pady=10)

# Create a button for image selection
button = ttk.Button(root, text="Choose the Image", command=choose_image, style='Custom.TButton')
button.pack(pady=20)

# Create a drag-and-drop frame
drag_drop_frame = tk.Frame(root, bg="#F0F0F0", relief="ridge", bd=2, height=100, width=400)
drag_drop_frame.pack_propagate(False)
drag_drop_frame.pack(pady=10)

drag_drop_label = tk.Label(drag_drop_frame, text="Or drag and drop PNG image here", font=("Helvetica", 12), bg="#F0F0F0", fg="#666")
drag_drop_label.pack(expand=True)

# Add a drop target for drag-and-drop functionality
drag_drop_frame.drop_target_register(DND_FILES)
drag_drop_frame.dnd_bind('<<Drop>>', on_drop)

# Create a frame for the prediction result
result_frame = tk.Frame(root, bg="#FFFFFF", relief="ridge", bd=2)
result_label = tk.Label(result_frame, text="", wraplength=500, font=("Helvetica", 14), bg="#FFFFFF", fg="#333")
result_label.pack(padx=10, pady=10)

# Add a "Back" button to reset the interface
back_button = ttk.Button(root, text="Back", command=go_back, style='Custom.TButton')
back_button.pack_forget()  # Initially hide the back button

# Footer frame
footer_frame = tk.Frame(root, bg="#F8F8F8")
footer_frame.pack(side="bottom", fill="x", pady=5)

# Footer content: copyright, GitHub link, version
copyright_label = tk.Label(
    footer_frame, 
    text="© 2025 Umar Robbani - Licensed under GPLv3", 
    font=("Helvetica", 12), 
    bg="#F8F8F8", 
    fg="#666"
)
copyright_label.pack(side="left", padx=10)

github_label = tk.Label(
    footer_frame, 
    text="GitHub", 
    font=("Helvetica", 12, "underline"), 
    bg="#F8F8F8", 
    fg="#1E90FF", 
    cursor="hand2"
)
github_label.pack(side="left", padx=10)
github_label.bind("<Button-1>", lambda e: open_github())

version_label = tk.Label(footer_frame, text="v1.0.0", font=("Helvetica", 12), bg="#F8F8F8", fg="#666")
version_label.pack(side="right", padx=10)


# Function to open GitHub link
def open_github():
    import webbrowser
    webbrowser.open_new("https://github.com/07umar07/Tuberculosis-Detector")

# Initially hide the frames
image_frame.pack_forget()
result_frame.pack_forget()

# Run the GUI event loop
root.mainloop()
